<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 0.11.25</div>
        </div>
    </div>
</div>

# Diffusion Equation

We consider a spatial domain $\Omega \subset \mathbf{R}^d$ (d = 2 or 3), a time interval $I = [0, T]$, and
a diffusion equation of the following form:

$$
  m_0(\mathbf{x}) \partial_t u(\mathbf{x},t) - \nabla \cdot \big(\mathcal{D}(\mathbf{x}) \nabla u(\mathbf{x},t)\big) = 0,
$$

with initial conditions

$$u(\mathbf{x},0) = u_{\text{init}}(\mathbf{x})$$.


Here, $u$ denotes the space- and time-dependent diffusive field and $u_{\text{init}}$ are describes external forces.
$\partial_t$ denotes the first time derivative and $\nabla$ the spatial gradient operator.
Furthermore, the scalar parameter $m_0$ and the symmetric second-order diffusion tensor $\mathcal{D}$ are space-dependent coefficients.

$\mathcal{D}$ can be related to a Wiener process using the relation

$$\mathcal{D} = \frac{1}{2} \sigma \sigma\,^T,$$

which direction-dependent smoothing lengths $\sigma_i$.

For the special case of $m_0 = 1$ and $T = 1$, $\sigma$ corresponds to the standard deviation of the Gaussian smoothing in meters.

In the isotropic case, $\mathcal{D}$ simplifies to a scalar value, in which case we may re-write the diffusion equation as

$$
  m_0(\mathbf{x}) \partial_t u(\mathbf{x},t) - \nabla \cdot \big(m_1(\mathbf{x}) \nabla u(\mathbf{x},t)\big) = 0,
$$

with $m_1 = 0.5 * \sigma^2$ and the isotropic smoothing length $\sigma$.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# Standard Python packages
import matplotlib.pyplot as plt
import numpy as np
import os
import toml

# Salvus imports
from salvus.mesh.structured_grid_2D import StructuredGrid2D
from salvus.mesh.unstructured_mesh import UnstructuredMesh
import salvus.flow.api
import salvus.flow.simple_config as sc

SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "token")

In [ ]:
sg = StructuredGrid2D.rectangle(nelem_x=40, nelem_y=60, max_x=4.0, max_y=6.0)
mesh = sg.get_unstructured_mesh()
mesh.find_side_sets("cartesian")
input_mesh = mesh.copy()

In [ ]:
input_mesh.attach_field("some_field", np.random.randn(mesh.npoint))
input_mesh.map_nodal_fields_to_element_nodal()
input_mesh.write_h5("initial_values.h5")
input_mesh

In [ ]:
smoothing_length_in_meters = 0.1

mesh.attach_field("M0", np.ones_like(mesh.get_element_nodes()[:, :, 0]))
mesh.attach_field(
    "M1",
    0.5
    * smoothing_length_in_meters ** 2
    * np.ones_like(mesh.get_element_nodes()[:, :, 0]),
)
mesh.attach_field("fluid", np.ones(mesh.nelem))
mesh

In [ ]:
sim = sc.simulation.Diffusion(mesh=mesh)

sim.domain.polynomial_order = 1

sim.physics.diffusion_equation.time_step_in_seconds = 1e-3
sim.physics.diffusion_equation.courant_number = 0.06

sim.physics.diffusion_equation.initial_values.filename = "initial_values.h5"
sim.physics.diffusion_equation.initial_values.format = "hdf5"
sim.physics.diffusion_equation.initial_values.field = "some_field"

sim.physics.diffusion_equation.final_values.filename = "out.h5"

sim.output.volume_data.filename = "diffusion.h5"
sim.output.volume_data.format = "hdf5"
sim.output.volume_data.fields = ["phi"]
sim.output.volume_data.sampling_interval_in_time_steps = 10

sim.validate()

In [ ]:
salvus.flow.api.run(
    site_name=SALVUS_FLOW_SITE_NAME,
    input_file=sim,
    ranks=1,
    output_folder="output",
    get_all=True,
    overwrite=True,
    wall_time_in_seconds=600,
)

In [ ]:
mesh = UnstructuredMesh.from_h5(filename="output/out.h5")
mesh